# Основы молекулярной динамики.
**3 вариант**

In [ ]:
import numpy as np
import plotly
import plotly.graph_objs as go
import plotly.express as px
from plotly.subplots import make_subplots
from tqdm import tqdm
import time

In [ ]:
# Число частиц
N = 60
# Размер двумерной области
x_size = 10
y_size = 10
# Масса одной частицы
m = 0.2
sigma = 0.03
epsilon = 0.05
r_min = 0.028
kBT = 0.1
# Число степеней свободы для частицы
f = 2
# Постоянная Больцмана
kB = 1
T = 0.1/kB
# сетка частиц
#x_values = np.linspace(0, x_size, int(np.sqrt(N)))
#y_values = np.linspace(0, y_size, int(np.sqrt(N)))
#pos = np.array(np.meshgrid(x_values, y_values)).T.reshape(-1, 2)[:N]
# рандомный сид, нормальное распределение
pos = np.random.random((N, 2)) * [x_size, y_size]
# рандомный сид, равномерное распределение
#pos = np.random.uniform(0, [x_size, y_size], size=(N, 2))
velocities = np.random.normal(0, np.sqrt(kBT / m), (N, 2))
velocities -= np.mean(velocities, axis=0)

In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=pos[:, 0], y=pos[:, 1], mode='markers'))
fig.update_layout(title="Расположение атомов в двумерной коробке")
fig.show()

In [ ]:
speed = np.linalg.norm(velocities, axis=1)

fig = px.histogram(speed, nbins=30, labels={'value': 'Скорость', 'count': 'Число частиц'}, title='Распределение частиц по скоростям', histnorm='percent')
fig.update_layout(xaxis_title='Скорость', yaxis_title='Процент частиц')
fig.show()

**Для увеличения точности во всех пунктах задания, необходимо уменьшить шаг dt**

# Пункт 1.0

In [ ]:
def build_cell_lists(positions, cell_size):
        n_cells_x = int(x_size / cell_size)
        n_cells_y = int(y_size / cell_size)
        cell_lists = [[] for _ in range(n_cells_x * n_cells_y)]
        for i in range(N):
            cell_x = int(positions[i, 0] / cell_size)
            cell_y = int(positions[i, 1] / cell_size)
            if 0 <= cell_x < n_cells_x and 0 <= cell_y < n_cells_y:
                cell_lists[cell_x + cell_y * n_cells_x].append(i)
        return cell_lists

def get_neighbor_list(particle_index, positions, cell_size, cell_lists):
        n_cells_x = int(x_size / cell_size)
        n_cells_y = int(y_size / cell_size)
        cell_x = int(positions[particle_index, 0] / cell_size)
        cell_y = int(positions[particle_index, 1] / cell_size)
        neighbors = []
        
        for dx in [-1, 0, 1]:
            for dy in [-1, 0, 1]:
                nx, ny = cell_x + dx, cell_y + dy
                if 0 <= nx < n_cells_x and 0 <= ny < n_cells_y:
                    neighbors.extend(cell_lists[nx + ny * n_cells_x])
        
        return neighbors

In [ ]:
def periodic_forces_calc(positions):
    force = np.zeros_like(positions)
    virial = 0.0
    #cell_lists = build_cell_lists(positions, cell_size)
    #for i in range(N):
        #neighbors = get_neighbor_list(i, positions, cell_size, cell_lists)
        #for j in neighbors:
            #if i == j:
                #continue
    for i in range(N):
        for j in range(i + 1, N):
            r_vec = (positions[i] - positions[j]) - np.round((positions[i] - positions[j]) / [x_size, y_size]) * [x_size, y_size]
            r = np.linalg.norm(r_vec)
            if r < r_min:
                f_mag = 4 * epsilon * ((12 * (sigma / r_min)**12) - (6 * (sigma / r_min)**6))
            else:
                f_mag = 4 * epsilon * ((12 * (sigma / r)**12) - (6 * (sigma / r)**6))
            force[i] += f_mag * (r_vec / r)
            force[j] -= f_mag * (r_vec / r)
            virial += np.dot(r_vec, (f_mag * (r_vec / r)))
    return force, virial

In [ ]:
def periodic_potential_energy(positions, forces):
    potential_energy = 0.0
    #cell_lists = build_cell_lists(positions, cell_size)
    #for i in range(N):
        #neighbors = get_neighbor_list(i, positions, cell_size, cell_lists)
        #for j in neighbors:
            #if i == j:
                #continue
    for i in range(N):
        for j in range(i + 1, N):
            r_vec = (positions[i] - positions[j]) - np.round((positions[i] - positions[j]) / [x_size, y_size]) * [x_size, y_size]
            r = np.linalg.norm(r_vec)
            if r < r_min:  
                potential_energy += ((4 * epsilon * ((sigma / r_min)**12 - (sigma / r_min)**6)) + np.dot(forces[i], forces[j])*(r_min-r))
            else:  
                potential_energy += 4 * epsilon * ((sigma / r)**12 - (sigma / r)**6)
    return potential_energy

In [ ]:
def periodic_virial_func(positions, forces):
    F_virial = 0.0
    for i in range(N):
        for j in range(i + 1, N):
            r_vec = (positions[i] - positions[j]) - np.round((positions[i] - positions[j]) / [x_size, y_size]) * [x_size, y_size]
            r = np.linalg.norm(r_vec)
            if r > 0: 
                F_virial += r * np.dot(forces[i], forces[j])  
    return F_virial

In [ ]:
def dynamics_animation(trajectory):
    frames = [
        go.Frame(
            data=[go.Scatter(x=tra[:, 0], y=tra[:, 1], mode='markers', marker=dict(size=10, color='blue'))]
        ) 
        for tra in trajectory
    ]

    initial_figure = go.Figure(
        data=[go.Scatter(
            x=trajectory[0][:, 0],
            y=trajectory[0][:, 1],
            mode='markers',
            marker=dict(size=10, color='blue')
        )]
    )

    initial_figure.update_layout(
        xaxis=dict(range=[-3, 3], autorange=False),
        yaxis=dict(range=[-3, 3], autorange=False),
        title='Particle Animation',
        showlegend=False,
        updatemenus=[
            dict(type='buttons',
                 showactive=False,
                 buttons=[
                     dict(label='Play', method='animate',
                          args=[None, dict(frame=dict(duration=100, redraw=True),
                                           transition=dict(duration=0),  # отключаем сглаживание
                                           fromcurrent=True,
                                           easing='linear')]),
                     dict(label='Pause', method='animate',
                          args=[[None], dict(frame=dict(duration=0, redraw=False),
                                             mode='immediate',
                                             transition=dict(duration=0))]),
                     dict(label='Rewind', method='animate',
                          args=[None, dict(frame=dict(duration=100, redraw=True),
                                           transition=dict(duration=0),
                                           mode='toback',
                                           easing='linear')])
                 ])
        ]
    )

    initial_figure.frames = frames

    initial_figure.show()

In [ ]:
instant_energies1 = []
instant_pressures1 = []
instant_temperatures1 = []
dt = 0.1
r_end = 2.5*sigma
# cell_size = 2*r_end
timesteps = np.arange(0, 1001, dt) 
velocities_1 = velocities.copy()
pos_1 = pos.copy()
V = x_size * y_size 
t = 0
trajectory = []
E_kin = np.sum(0.5 * m * velocities_1**2)
temperature = (1 / (N * f * kB)) * E_kin
start_time = time.time()

for t in tqdm(timesteps, desc="Simulation"):
    trajectory.append(pos_1.copy())
    forces, virial = periodic_forces_calc(pos_1)
    a = forces / m
    velocities_i1 = velocities_1 + a * dt
    
    # Термостат Берендсена
    scale = np.sqrt(1 + (dt / 0.1) * (T / temperature - 1))
    velocities_i1 *= scale
    
    pos_i1 = pos_1 + ((velocities_1 + velocities_i1) / 2) * dt
    pos_i1 %= np.array([x_size, y_size])
    
    E_kin = np.sum(0.5 * m * velocities_i1**2)
    temperature = (1 / (N * f * kB)) * E_kin
    

    if t % 100 == 0:
        E_pot = periodic_potential_energy(pos_i1, forces)
        pressure = (1 / (V * 3)) * ((E_kin * 2) + virial)
        energy = E_pot + E_kin
        instant_pressures1.append(pressure)
        instant_temperatures1.append(temperature)
        instant_energies1.append(energy)
    
    velocities_1 = velocities_i1
    pos_1 = pos_i1
end_time = time.time()
elapsed_time = end_time - start_time
print('Время расчета: ', elapsed_time, ' секунд')
timesteps_filtered = np.arange(0, 1001, 100)
fig1 = go.Figure()
fig1.add_trace(go.Scatter(x=timesteps_filtered, y=instant_temperatures1, mode='markers'))
fig1.update_layout(title="График мгновенных температур с периодическими граничными условиями",
                  xaxis_title="Момент времени",
                  yaxis_title="Температура, К")
fig1.show()
fig2 = go.Figure()
fig2.add_trace(go.Scatter(x=timesteps_filtered, y=instant_pressures1, mode='markers'))
fig2.update_layout(title="График мгновенных давлений с периодическими граничными условиями",
                  xaxis_title="Момент времени",
                  yaxis_title="Давление, Па")
fig2.show()
fig3 = go.Figure()
fig3.add_trace(go.Scatter(x=timesteps_filtered, y=instant_energies1, mode='markers'))
fig3.update_layout(title="График мгновенных энергий с периодическими граничными условиями",
                  xaxis_title="Момент времени",
                  yaxis_title="Энергия, Дж")
fig3.show()
dynamics_animation(trajectory)

# Пункт 2.0

In [ ]:
def periodic_forces_calc_LJTS(positions, r_end):
    force = np.zeros_like(positions)
    virial = 0.0
    for i in range(N):
        for j in range(i + 1, N):
            r_vec = (positions[i] - positions[j]) - np.round(positions[i] - positions[j] / [x_size, y_size]) * [x_size, y_size]
            r = np.linalg.norm(r_vec)
            if r < r_min: 
                f_mag = (4 * epsilon * ((12 * (sigma / r_min)**12) - (6 * (sigma / r_min)**6)))
            elif r < r_end:
                f_mag = (4 * epsilon * ((12 * (sigma / r)**12) - (6 * (sigma / r)**6)))
            else:
                continue
            force[i] += f_mag * (r_vec / r) 
            force[j] -= f_mag * (r_vec / r)  
            virial += np.dot(r_vec, (f_mag * (r_vec / r)))
    return force, virial


In [ ]:
def periodic_potential_energy_LJTS(positions, r_end, forces):
    energy = 0.0
    energy_end = 4 * epsilon * ((sigma / r_end)**12 - (sigma / r_end)**6)
    
    for i in range(N):
        for j in range(i + 1, N):
            r_vec = (positions[i] - positions[j]) - np.round(positions[i] - positions[j] / [x_size, y_size]) * [x_size, y_size] # периодические граничные условия
            r = np.linalg.norm(r_vec) 
            if r < r_min: 
                energy += (((4 * epsilon * ((sigma / r_min)**12 - (sigma / r_min)**6)) + np.dot(forces[i], forces[j])*(r_min-r)) - energy_end)
            elif r < r_end:
                energy += (4 * epsilon * ((sigma / r)**12 - (sigma / r)**6) - energy_end)
            else:
                continue
            
    return energy

In [ ]:
instant_energies2 = []
instant_pressures2 = []
instant_temperatures2 = []
dt = 0.1
r_end = 2.5*sigma
timesteps = np.arange(0, 1001, dt) 
velocities_2 = velocities.copy()
pos_2 = pos.copy()
V = x_size * y_size
trajectory = []

E_kin = np.sum(0.5 * m * velocities_2**2)
temperature = (1 / (N * f * kB)) * E_kin
start_time = time.time()

for t in tqdm(timesteps, desc="Simulation"):
    trajectory.append(pos_2.copy())
    forces, virial = periodic_forces_calc_LJTS(pos_2, r_end)
    a = forces / m
    velocities_i2 = velocities_2 + a * dt
    
    # Термостат Берендсена
    scale = np.sqrt(1 + (dt / 0.1) * (T / temperature - 1))
    velocities_i2 *= scale
    
    pos_i2 = pos_2 + ((velocities_2 + velocities_i2) / 2) * dt
    pos_i2 %= np.array([x_size, y_size])
    
    E_kin = np.sum(0.5 * m * velocities_i2**2)
    temperature = (1 / (N * f * kB)) * E_kin
    
    if t % 100 == 0:
        E_pot = periodic_potential_energy_LJTS(pos_i2, r_end, forces)
        pressure = (1 / (V * 3)) * ((E_kin * 2) + virial)  
        energie = E_kin + E_pot
        instant_pressures2.append(pressure)
        instant_temperatures2.append(temperature)
        instant_energies2.append(energie)

    velocities_2 = velocities_i2
    pos_2 = pos_i2

end_time = time.time()
elapsed_time = end_time - start_time
print('Время расчета: ', elapsed_time, ' секунд')
timesteps_filtered = np.arange(0, 1001, 100)
fig1 = go.Figure()
fig1.add_trace(go.Scatter(x=timesteps_filtered, y=instant_temperatures2, mode='markers'))
fig1.update_layout(title="График мгновенных температур с периодическими граничными условиями и LJTS взаимодействием",
                  xaxis_title="Момент времени",
                  yaxis_title="Температура, К")
fig1.show()
fig2 = go.Figure()
fig2.add_trace(go.Scatter(x=timesteps_filtered, y=instant_pressures2, mode='markers'))
fig2.update_layout(title="График мгновенных давлений с периодическими граничными условиями и LJTS взаимодействием",
                  xaxis_title="Момент времени",
                  yaxis_title="Давление, Па")
fig2.show()
fig3 = go.Figure()
fig3.add_trace(go.Scatter(x=timesteps_filtered, y=instant_energies2, mode='markers'))
fig3.update_layout(title="График мгновенных энергий с периодическими граничными условиями и LJTS взаимодействием",
                  xaxis_title="Момент времени",
                  yaxis_title="Энергия, Дж")
fig3.show()
dynamics_animation(trajectory)

# Пункт 3.0

In [ ]:
def mirror_positions(positions):
    # Отражает позиции частиц в случае выхода за границы.
    positions[positions < 0] = -positions[positions < 0]

    out_of_bounds_x = positions[:, 0] > x_size
    out_of_bounds_y = positions[:, 1] > y_size
    
    positions[out_of_bounds_x, 0] = 2 * x_size - positions[out_of_bounds_x, 0]
    positions[out_of_bounds_y, 1] = 2 * y_size - positions[out_of_bounds_y, 1]

    return positions



In [ ]:
def mirror_velocities(positions, velocities):
    # Отражает вектор скорости.
    velocities[positions[:, 0] < 0, 0] *= -1
    velocities[positions[:, 0] > x_size, 0] *= -1

    velocities[positions[:, 1] < 0, 1] *= -1
    velocities[positions[:, 1] > y_size, 1] *= -1

    return velocities

In [ ]:
def mirror_forces(positions, forces):
    # Отражает вектор силы.
    forces[positions[:, 0] < 0, 0] *= -1
    forces[positions[:, 0] > x_size, 0] *= -1

    forces[positions[:, 1] < 0, 1] *= -1
    forces[positions[:, 1] > y_size, 1] *= -1

    return forces

In [ ]:
def mirror_virial_func(positions, forces):
    F_virial = 0.0
    for i in range(N):
        for j in range(i + 1, N):
            r_vec = (positions[i] - positions[j])
            r = np.linalg.norm(r_vec)
            if r > 0: 
                F_virial += r * np.dot(forces[i], forces[j])   
    return F_virial

In [ ]:
def potential_energy(positions, r_end, forces):
    energy = 0.0
    energy_end = 4 * epsilon * ((sigma / r_end)**12 - (sigma / r_end)**6)
    
    for i in range(N):
        for j in range(i + 1, N):
            r_vec = positions[i] - positions[j]
            r = np.linalg.norm(r_vec)
            if r < r_min: 
                energy += (((4 * epsilon * ((sigma / r_min)**12 - (sigma / r_min)**6)) + np.dot(forces[i], forces[j])*(r_min-r)) - energy_end)
            elif r < r_end:
                energy += (4 * epsilon * ((sigma / r)**12 - (sigma / r)**6) - energy_end)
            else:
                continue
    
    return energy

In [ ]:
def forces_calc(positions, r_end):
    force = np.zeros_like(positions)
    virial = 0.0
    for i in range(N):
        for j in range(i + 1, N):
            r_vec = (positions[i] - positions[j])
            r = np.linalg.norm(r_vec)
            if r < r_min: 
                f_mag = (4 * epsilon * ((12 * (sigma / r_min)**12) - (6 * (sigma / r_min)**6)))
            elif r < r_end:
                f_mag = (4 * epsilon * ((12 * (sigma / r)**12) - (6 * (sigma / r)**6)))
            else:
                continue
            force[i] += f_mag * (r_vec / r) 
            force[j] -= f_mag * (r_vec / r)  
            virial += np.dot(r_vec, (f_mag * (r_vec / r)))
    return force, virial

In [ ]:
instant_energies3 = []
instant_pressures3 = []
instant_temperatures3 = []
dt = 0.1
r_end = 2.5*sigma
timesteps = np.arange(0, 1001, dt) 
velocities_3 = velocities.copy()
pos_3 = pos.copy()
V = x_size * y_size
trajectory = []

E_kin = np.sum(0.5 * m * velocities_3**2)
temperature = (1 / (N * f * kB)) * E_kin

start_time = time.time()

for t in tqdm(timesteps, desc="Simulation"):
    trajectory.append(pos_3.copy())
    forces, virial = forces_calc(pos_3, r_end)
    forces = mirror_forces(pos_3, forces)  

    a = forces / m
    velocities_i3 = velocities_3 + a * dt
    
    # Термостат Берендсена
    scale = np.sqrt(1 + (dt / 0.1) * (T / temperature - 1))
    velocities_i3 *= scale

    pos_i3 = pos_3 + ((velocities_3 + velocities_i3) / 2) * dt
    
    velocities_i3 = mirror_velocities(pos_i3, velocities_i3)
    
    pos_3 = mirror_positions(pos_i3)
    
    E_kin = np.sum(0.5 * m * velocities_i3**2)
    temperature = (1 / (N * f * kB)) * E_kin
    
    if t % 100 == 0:
        E_pot = potential_energy(pos_3, r_end, forces)
        pressure = (1 / (V * 3)) * ((E_kin * 2) + virial)
        energie = E_pot + E_kin
        instant_pressures3.append(pressure)
        instant_temperatures3.append(temperature)
        instant_energies3.append(energie)

    velocities_3 = velocities_i3
    
end_time = time.time()
elapsed_time = end_time - start_time
print('Время расчета: ', elapsed_time, ' секунд')
timesteps_filtered = np.arange(0, 1001, 100)
fig1 = go.Figure()
fig1.add_trace(go.Scatter(x=timesteps_filtered, y=instant_temperatures3, mode='markers'))
fig1.update_layout(title="График мгновенных температур с зеркальными граничными условиями и LJTS взаимодействием",
                  xaxis_title="Момент времени",
                  yaxis_title="Температура, К")
fig1.show()
fig2 = go.Figure()
fig2.add_trace(go.Scatter(x=timesteps_filtered, y=instant_pressures3, mode='markers'))
fig2.update_layout(title="График мгновенных давлений с зеркальными граничными условиями и LJTS взаимодействием",
                  xaxis_title="Момент времени",
                  yaxis_title="Давление, Па")
fig2.show()
fig3 = go.Figure()
fig3.add_trace(go.Scatter(x=timesteps_filtered, y=instant_energies3, mode='markers'))
fig3.update_layout(title="График мгновенных энергий с зеркальными граничными условиями и LJTS взаимодействием",
                  xaxis_title="Момент времени",
                  yaxis_title="Энергия, Дж")
fig3.show()
dynamics_animation(trajectory)

# Пункт 4.0

In [ ]:
def modified_potential_energy(positions, r_end, forces):
    energy = 0.0
    energy_end = 4 * epsilon * ((sigma / r_end)**12 - (sigma / r_end)**6)
    for i in range(N):
        for j in range(i + 1, N):
            r_vec = positions[i] - positions[j]
            r_vec[1] -= y_size * np.round(r_vec[1] / y_size)  # Периодические условия по оси y
            r = np.linalg.norm(r_vec)
            if r < r_min:  
                energy += (((4 * epsilon * ((sigma / r_min)**12 - (sigma / r_min)**6)) + np.dot(forces[i], forces[j])*(r_min-r)) - energy_end)
            elif r < r_end:
                energy += (4 * epsilon * ((sigma / r)**12 - (sigma / r)**6) - energy_end)
            else:
                continue
        # Левая стенка  
        if positions[i, 0] < r_end and positions[i, 0] > r_min: 
            r = positions[i, 0]
            energy += (4 * epsilon * ((sigma / r)**12 - (sigma / r)**6) - energy_end)
        elif positions[i, 0] < r_end and positions[i, 0] < r_min:
            energy += (4 * epsilon * ((sigma / r_min)**12 - (sigma / r_min)**6) + (4 * epsilon * ((12 * (sigma / r_min)**12) - (6 * (sigma / r_min)**6))*(r_min-r)) - energy_end)
        # Правая стенка   
        elif positions[i, 0] > x_size - r_end and positions[i, 0] < x_size - r_min:
            r = x_size - positions[i, 0]
            energy += (4 * epsilon * ((sigma / r)**12 - (sigma / r)**6) - energy_end)
        elif positions[i, 0] > x_size - r_end and positions[i, 0] > x_size - r_min:
            energy += (4 * epsilon * ((sigma / r_min)**12 - (sigma / r_min)**6) + (4 * epsilon * ((12 * (sigma / r_min)**12) - (6 * (sigma / r_min)**6))*(r_min-r)) - energy_end)
        else:
            continue
    return energy

In [ ]:
def modified_forces_calc(positions, r_end):
    force = np.zeros_like(positions)
    virial = 0.0  

    for i in range(N):
        for j in range(i + 1, N):
            r_vec = positions[i] - positions[j]
            r_vec[1] -= y_size * np.round(r_vec[1] / y_size)  # Периодические условия по оси y
            r = np.linalg.norm(r_vec)
            if r < r_min:
                f_mag = 4 * epsilon * (12 * (sigma / r_min)**12 - 6 * (sigma / r_min)**6)
            elif r < r_end:
                f_mag = 4 * epsilon * (12 * (sigma / r)**12 - 6 * (sigma / r)**6)
            else:
                continue

            f_vec = f_mag * (r_vec / r)
            force[i] += f_vec
            force[j] -= f_vec

            virial += np.dot(r_vec, f_vec)

    
        f_mag_x = 0.0
        # Левая стенка
        if positions[i, 0] < r_end and positions[i, 0] > r_min:
            r = positions[i, 0]
            f_mag_x = 4 * epsilon * (12 * (sigma / r)**12 - 6 * (sigma / r)**6)
            force[i, 0] += f_mag_x
            virial += positions[i, 0] * f_mag_x  # r_x * F_x

        elif positions[i, 0] < r_end and positions[i, 0] <= r_min:
            r = r_min
            f_mag_x = 4 * epsilon * (12 * (sigma / r)**12 - 6 * (sigma / r)**6)
            force[i, 0] += f_mag_x
            virial += positions[i, 0] * f_mag_x

        # Правая стенка
        elif positions[i, 0] > x_size - r_end and positions[i, 0] < x_size - r_min:
            r = x_size - positions[i, 0]
            f_mag_x = 4 * epsilon * (12 * (sigma / r)**12 - 6 * (sigma / r)**6)
            force[i, 0] -= f_mag_x
            virial += (positions[i, 0] - x_size) * (-f_mag_x)  # r_x * F_x

        elif positions[i, 0] >= x_size - r_min:
            r = r_min
            f_mag_x = 4 * epsilon * (12 * (sigma / r)**12 - 6 * (sigma / r)**6)
            force[i, 0] -= f_mag_x
            virial += (positions[i, 0] - x_size) * (-f_mag_x)

    return force, virial

In [ ]:
def modified_virial_func(positions, forces):
    F_virial = 0.0
    for i in range(N):
        for j in range(i + 1, N):
            r_vec = (positions[i] - positions[j])
            r_vec[1] -= y_size * np.round(r_vec[1] / y_size)
            r = np.linalg.norm(r_vec)
            if r > 0: 
                F_virial += r * np.dot(forces[i], forces[j]) 
    return F_virial

In [ ]:
instant_energies4 = []
instant_pressures4 = []
instant_temperatures4 = []
dt = 0.05
r_end = 2.5 * sigma
timesteps = np.arange(0, 1001, dt)
velocities_4 = velocities.copy()
pos_4 = pos.copy()
V = x_size * y_size
trajectory = []

E_kin = np.sum(0.5 * m * velocities_4**2)
temperature = (1 / (N * f * kB)) * E_kin

start_time = time.time()

for t in tqdm(timesteps, desc="Simulation"):
    trajectory.append(pos_4.copy())
    forces, virial = modified_forces_calc(pos_4, r_end)
    a = forces / m
    velocities_i4 = velocities_4 + a * dt
    
    # Термостат Берендсена
    scale = np.sqrt(1 + (dt / 0.1) * (T / temperature - 1))
    velocities_i4 *= scale
    
    pos_i4 = pos_4 + ((velocities_4 + velocities_i4) / 2) * dt
    
    # Периодические условия по оси y
    pos_i4[:, 1] %= y_size
    # Искусственные стенки по оси x
    pos_i4[:, 0] = np.clip(pos_i4[:, 0], 0.01, x_size-0.01)
    
    E_kin = np.sum(0.5 * m * velocities_i4**2)
    temperature = (1 / (N * f * kB)) * E_kin
    
    if t % 100 == 0:
        E_pot = modified_potential_energy(pos_i4, r_end, forces)
        pressure = (1 / (V * 3)) * ((E_kin * 2) + virial)
        energie = E_pot + E_kin
        instant_pressures4.append(pressure)
        instant_temperatures4.append(temperature)
        instant_energies4.append(energie)
    
    velocities_4 = velocities_i4
    pos_4 = pos_i4

end_time = time.time()
elapsed_time = end_time - start_time
print('Время расчета: ', elapsed_time, ' секунд')

timesteps_filtered = np.arange(0, 1001, 100)
fig1 = go.Figure()
fig1.add_trace(go.Scatter(x=timesteps_filtered, y=instant_temperatures4, mode='markers'))
fig1.update_layout(title="График мгновенных температур потенциалом LJTS на стенках по оси x и периодическими условиями по оси y",
                  xaxis_title="Момент времени",
                  yaxis_title="Температура, К")
fig1.show()

fig2 = go.Figure()
fig2.add_trace(go.Scatter(x=timesteps_filtered, y=instant_pressures4, mode='markers'))
fig2.update_layout(title="График мгновенных давлений с потенциалом LJTS на стенках по оси x и периодическими условиями по оси y",
                  xaxis_title="Момент времени",
                  yaxis_title="Давление, Па")
fig2.show()
fig3 = go.Figure()
fig3.add_trace(go.Scatter(x=timesteps_filtered, y=instant_energies4, mode='markers'))
fig3.update_layout(title="График мгновенных энергий с потенциалом LJTS на стенках по оси x и периодическими условиями по оси y",
                  xaxis_title="Момент времени",
                  yaxis_title="Энергия, Дж")
fig3.show()
dynamics_animation(trajectory)